In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
import tensorflow.keras as keras
from noise_generator.noisegen import NoiseHandling
from morse_label_funcs import code_number
import time
from audio_process.fft import expand_image_dims, train_img_generate, SIGNAL_TO_PIXEL_CONSTANT
from Image_Generator_helpers import  DataSets, set_paths, global_path
from data_filters import min_n_letters_raw, position_labels_de_normalizer

Models

In [ ]:

model_velocity = keras.models.load_model('saved_model_velocity_raw')
model_regression = keras.models.load_model('saved_model_regresssion_raw_low_tempo_5')
model_categorical = keras.models.load_model('saved_model_categorical_right_align_low_tempo')
model_binary = keras.models.load_model('saved_model_raw_binary_4')

Constants

In [ ]:
TRAINING_IMAGE_DIMENSIONS = (5, 1400)
NODGE_IMAGE_PIXEL_AMOUNT = 6 #Push the image to the left to adjust for incorrect position prediction
CATEGORICAL_IMAGE_CROPPED_WIDTH = TRAINING_IMAGE_DIMENSIONS[1] - 1250 #Cropped with during traing of categorical prediction model"
FFT_JUMP = 64
MEAN_TEMPO_OF_TRAINING_DATA = (16 + 22) / 2 # The mean tempo of the training data tempis 
SIGNAL_LENGTH_MULTIPLIER: int = 200
SIGNAL_LENGTH =  SIGNAL_TO_PIXEL_CONSTANT * SIGNAL_LENGTH_MULTIPLIER
ALICE_TEXT = "Alice was beginning to get very tired of sitting by her sister on the bank and of having nothing to do once or twice she had peeped into the book her sister was reading but it had no pictures or conversations in it and what is the use of a book thought Alice without pictures or conversations So she was considering in her own mind as well as she could for the hot day made her feel very sleepy and stupid whether the pleasure of making a daisy chain would be worth the trouble of getting up and picking the daisies when suddenly a White Rabbit with pink eyes ran close by her"
ALICE_CORRECT: str = ALICE_TEXT.strip().replace(" ", "").lower()

Data

In [ ]:
ORIG_AUDIO_FILE_NAME = 'training_data/MorseTrainSet_23/AUDIO23/BOPAEWITAVZSEE_10400_23_010.wav'
ORIG_AUDIO_FILE_NAME_2 = 'training_data/MorseTrainSet_23/AUDIO23/ZBGLAQEMPZZNBNA_14400_23_021.wav'
ALICE = 'Alice_audio/Alice_30_000.wav'
ALICE_001 = 'Alice_audio/Alice_30_001.wav'
ALICE_002 = 'Alice_audio/Alice_30_002.wav'
ALICE_020 = 'Alice_audio/Alice_30_020.wav'
ALICE_021 = 'Alice_audio/Alice_30_012.wav'
ALICE_022 = 'Alice_audio/Alice_30_022.wav'

Functions

In [ ]:

noise_handling = NoiseHandling()
noise_handling.SetFrequencies(6000,2200,200)
# From 0 = no noise to -15 = significant noise
# INFO:  table of SNRdb vs. digit in version\n",
# ver \t 0   1   2  3  4\n",
# SNRdB   30  10  5  2  0\n",

def apply_noise(signal, signal_to_noise_ratio_db):
    signal_noise, some_noise = noise_handling.addNoise(signal, signal_to_noise_ratio_db)
    return signal_noise

# insert an image into an empty image at start and en position
def insert_image(empty_image, image, start_pos, end_pos):
    empty_image[:, start_pos:end_pos] = image
    return empty_image

def predict_velocity(img):
    return (model_velocity(expand_image_dims(img))).numpy()[0][0] * 100

def rescale_fft(velocity_prediction, fft_jump, mean_tempo_of_training_velocity):
    return fft_jump * (mean_tempo_of_training_velocity / velocity_prediction)

def left_shift_image(img, shift):
    shift = int(shift)
    shifted_left = np.pad(img, [(0,0),(0,abs(shift))], mode='constant')[:, abs(shift): img.shape[1] + abs(shift)]
    return shifted_left

def show_image(img, width, position = [], title=""):
    plt.figure(figsize=(30,5))
    plt.xlim(0, width)
    if len(position) != 0:
        plt.xticks(position)    
    plt.imshow(img)
    plt.title(title, fontsize=18)
    plt.show()

def get_signal(wav):
    SampleRate, signal = wavfile.read(wav)
    # length = signal.shape[0] / SampleRate

    signal = np.float32(signal)
    signal = signal / np.max(signal)
    return signal


In [ ]:
signal = get_signal(ALICE)

In [ ]:


signal_to_noise_ratio_db = 3 # From 0 = no noise to -15 = significant noise
print("Noise level: ", signal_to_noise_ratio_db)
signal_noise = apply_noise(signal, signal_to_noise_ratio_db)
img_noise = train_img_generate(signal_noise, FFT_JUMP, SIGNAL_LENGTH)
velocity_prediction = predict_velocity(img_noise[:, :1400])
print("Velocity prediction: " , velocity_prediction)
rescaled_fft_jump = rescale_fft(velocity_prediction, FFT_JUMP, MEAN_TEMPO_OF_TRAINING_DATA)
img_noise_rescaled = train_img_generate(signal_noise, rescaled_fft_jump, SIGNAL_LENGTH)

rescaled_velocity_prediction = predict_velocity(img_noise_rescaled[:, :1400])
print("Rescaled Velocity prediction: ", rescaled_velocity_prediction)

show_image(img_noise_rescaled, 200, title="Starting image...")


In [ ]:

print("Correct: ", ALICE_CORRECT)
print("Correct length: ", len(ALICE_CORRECT))

signal_not_found_count: int = 0
letters = []
signal_presence_images = []
img_noise_rescaled_images = []
image_with_categorical_cropped_images = []
DEBUG = True

def letter_sequence(img_noise_rescaled):

    img_noise_rescaled_images.append(img_noise_rescaled)
    signal_presence_image = img_noise_rescaled[:, :400]
    signal_presence_images.append(signal_presence_image)
    signal_presence = model_binary(expand_image_dims(signal_presence_image)).numpy()[0][0]

    if (signal_presence < 0.5):
        global signal_not_found_count
        signal_not_found_count += 1 
        if (signal_not_found_count > 15): # return if signal not found after n times
            return False

        img_noise_rescaled = left_shift_image(img_noise_rescaled, 45)
        return letter_sequence(img_noise_rescaled)
       
    signal_not_found_count = 0 # reset signal not found counter
    first_letter_position = model_regression(expand_image_dims(img_noise_rescaled[:,:400])).numpy()[0][0]
    first_letter_position = position_labels_de_normalizer(first_letter_position)
    first_letter_position_nodged = first_letter_position + NODGE_IMAGE_PIXEL_AMOUNT

    start_position = int(CATEGORICAL_IMAGE_CROPPED_WIDTH - first_letter_position_nodged)

    # Create the categorical prediction image
    image_with_categorical_cropped = insert_image(
        empty_image=np.zeros([5,CATEGORICAL_IMAGE_CROPPED_WIDTH]), 
        image=img_noise_rescaled[:,:int(first_letter_position_nodged)], 
        start_pos=start_position, 
        end_pos= int(first_letter_position_nodged) + start_position
    )

    image_with_categorical_cropped_images.append(image_with_categorical_cropped)
    categorical_prediction = model_categorical(expand_image_dims(image_with_categorical_cropped))
    letters.append(code_number[np.argmax(categorical_prediction)])

    
    if (DEBUG == True):
        list_length = len(letters) -1
        prediction_current_letter = letters[list_length]
        correct_current_letter = list(ALICE_CORRECT)[list_length]
        if (prediction_current_letter  !=  correct_current_letter): # Debug when failed
            print("list position: ", list_length)
            print("Current prediction progress: ", "". join(letters))
            title = f'Categorical : Prediction - {prediction_current_letter} - Correct: {correct_current_letter}'
            show_image(image_with_categorical_cropped, 150, title=title)
            show_image(img_noise_rescaled_images[-1], 400, title="Current main image")
            show_image(img_noise_rescaled_images[-2], 400, title="Previous main image")
            show_image(image_with_categorical_cropped_images[-2], 150, title="Categorical previous")
            return False
   
    shifted_image = left_shift_image(img_noise_rescaled, int(first_letter_position_nodged))

    return letter_sequence(shifted_image)


start_time = time.time()
letter_sequence(img_noise_rescaled)
elapsed_time = time.time() - start_time
print("Total time (seconds): ", elapsed_time)

prediction_joined = "". join(letters)
print("Prediction: ", prediction_joined)
print("Prediction length: ", len(prediction_joined))

print("Prediction success: ", prediction_joined.strip() == ALICE_CORRECT.strip())